##### Copyright 2022 The Cirq Developers

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Installing Cirq

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://quantumai.google/cirq/transform"><img src="https://quantumai.google/site-assets/images/buttons/quantumai_logo_1x.png" />View on QuantumAI</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/quantumlib/Cirq/blob/master/docs/transform.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/colab_logo_1x.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/quantumlib/Cirq/blob/master/docs/transform.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/github_logo_1x.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/Cirq/docs/transform.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/download_icon_1x.png" />Download notebook</a>
  </td>
</table>

In [ ]:
try:
    import cirq
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq
    import cirq
    print("installed cirq.")

In [ ]:
from typing import Optional

# What is a Transformer?
Simply put, a transformer is just a function that *transforms* an input circuit into an output circuit. The precide API for what qualifies as a valid cirq transformer will be defined later in the tutorial. 

Circuit transformations are often necessary to compile a user-defined circuit to an equivalent circuit which can be executed on a specific device or simulator. The compilation process often involves steps like:
- Gate Decompositions: Rewrite the circuit using only gates that belong to the device target gateset, i.e. set of gates which the device can execute. 
- Qubit Mapping and Routing: Map the logic qubits in the input circuit to physical qubits on the device and insert appropriate swap operations such that final circuit respects the hardware topology. 
- Circuit Optimizations: Perform hardware specific optimizations, like merging and replacing connected components of 1 and 2 operations with more efficient rewrites, commuting Z gates through the circuit, aligning gates in moments etc.


Cirq provides many out-of-the-box transformers which can be used as individual compilation passes and also provides a general framework for users to create their own transformers using powerful primitives and bundle a bunch of transformers together to enable compiling circuits for specific targets. 

# Transformer API

## `cirq.TRANSFORMER` contract and `@cirq.transformer` decorator.
Any callable that satisfies the `cirq.TRANSFORMER` contract, i.e. takes a `cirq.AbstractCircuit` and `cirq.TransformerContext` and returns a transformed `cirq.AbstractCircuit`, is a valid transformer in Cirq. 

You can create a custom transformer by simply decorating a class/method, that satisfies the above contract, with `@cirq.transformer` decorator. 


In [ ]:
@cirq.transformer
def reverse_circuit(circuit: 'cirq.AbstractCircuit', *, context: Optional[cirq.TransformerContext] = None) -> 'cirq.AbstractCircuit':
  """Transformer to reverse the input circuit."""
  return circuit[::-1]


@cirq.transformer
class SubstituteGate:
  """Transformer to substitute `source` gates with `target` in the input circuit."""
  def __init__(self, source: 'cirq.Gate', target: 'cirq.Gate'):
    self._source = source
    self._target = target

  def __call__(self, circuit: 'cirq.AbstractCircuit', *, context: Optional[cirq.TransformerContext] = None) -> 'cirq.Circuit':
    batch_replace = []
    for i, op in circuit.findall_operations(lambda op: op.gate == self._source):
      batch_replace.append((i, op, self._target.on(*op.qubits)))
    transformed_circuit = circuit.unfreeze(copy=True)
    transformed_circuit.batch_replace(batch_replace)
    return transformed_circuit

q = cirq.NamedQubit("q")
circuit = cirq.Circuit(cirq.X(q), cirq.CircuitOperation(cirq.FrozenCircuit(cirq.X(q), cirq.Y(q))), cirq.Z(q))
substitute_gate = SubstituteGate(cirq.X, cirq.S)
print("Original Circuit:", circuit, "\n", sep="\n")
print("Reversed Circuit:", reverse_circuit(circuit), "\n", sep="\n")
print("Substituted Circuit:", substitute_gate(circuit), sep="\n")

## `cirq.TransformerContext` to store common configurable options. 
You might be wondering what's the benefit of adding the `@cirq.transformer` decorator on the methods / classes as defined above? Does it add any extra functionality to the decorated objects at all? 

Yes! The transformer decorator inspects the `cirq.TransformerContext` argument and automatically appends useful functionality like support for automated logging and recursively running the transformer on nested sub-circutis. 


### `cirq.TransformerLogger` and support for automated logging.
The `cirq.TransformerLogger` class is used to log the action of a transformer on an input circuit. Decorating a method / class with the `@cirq.transformer` decorator automatically adds support for logging. 

You can log the action of running a series of transformers on an input circuit by sharing the same logger instance across transformer calls. 

This is very useful to inspect the actions of individual transformers on the input circuit when running transformer bundles like `cirq.optimize_for_target_gateset`.

In [ ]:
context = cirq.TransformerContext(logger=cirq.TransformerLogger())
transformed_circuit = reverse_circuit(circuit, context=context)
transformed_circuit = substitute_gate(transformed_circuit, context=context)
context.logger.show()

### Built-in support for `deep=True`.
Notice that the custom transformers used above only modified the top-level circuit and had no impact on sub-circuits wrapped inside `cirq.CircuitOperation`.

You can call `@cirq.transformer(add_deep_support=True)` to automatically add the functionality of recursively running the custom transformer on circuits wrapped inside `cirq.CircuitOperation`. The recursive execution behavior of the transformer can then be controlled by setting `deep=True` in the transformer context. 

In [ ]:
@cirq.transformer(add_deep_support=True)
def reverse_circuit_deep(circuit: 'cirq.AbstractCircuit', *, context: Optional[cirq.TransformerContext] = None) -> 'cirq.AbstractCircuit':
  """Transformer to reverse the input circuit."""
  return circuit[::-1]


@cirq.transformer(add_deep_support=True)
class SubstituteGateDeep(SubstituteGate):
  """Transformer to substitute `source` gates with `target` in the input circuit."""
  pass

context = cirq.TransformerContext(deep=True)
substitute_gate_deep = SubstituteGateDeep(cirq.X, cirq.S)
print("Original Circuit:", circuit, "\n", sep="\n")
print("Reversed Circuit with deep=True:", reverse_circuit_deep(circuit, context=context), "\n", sep="\n")
print("Substituted Circuit with deep=True:", substitute_gate_deep(circuit, context=context), sep="\n")

# Transformer Primitives and Decompositions

Cirq provides many useful primitives which can be composed together to build powerful transformers. In this section, we will highlight some of these tools. 


## Moment preserving transformer primitives

Cirq provides useful abstractions to implement common transformer patterns, while preserving the moment structure of input circuit. Some of the notable transformer primitives are:

- **`cirq.map_operations`**: Applies local transformations on operations, by calling `map_func(op)` for each `op`.
- **`cirq.map_moments`**: Applies local transformation on moments, by calling `map_func(m)` for each moment `m`.
- **`cirq.merge_operations`**: Merges connected component of operations by calling `merge_func(op1, op2)` on iteratively for every mergeable operation `op1` and `op2`.
- **`cirq.merge_moments`**: Merges adjacent moments, from left to right, by calling `merge_func(m1, m2)`.


An important property of these primitives is that they have support for common configurable options present in `cirq.TransformerContext`, such as `tags_to_ignore` and `deep`. See the example below for a better understanding.

In [ ]:
@cirq.transformer
def substitute_gate_using_primitives(
    circuit: 'cirq.AbstractCircuit', 
    *, 
    context: Optional['cirq.TransformerContext'] = None,
    source: 'cirq.Gate' = cirq.X,
    target: 'cirq.Gate' = cirq.S,
    ) -> 'cirq.Circuit':
  """Transformer to substitute `source` gates with `target` in the input circuit.
  
  The transformer is implemented using `cirq.map_operations` primitive and hence
  has built-in support for 
    1. Recursively running the transformer on sub-circuits if `context.deep is True`.
    2. Ignoring operations tagged with any of `context.tags_to_ignore`. 
  """
  return cirq.map_operations(
      circuit, 
      map_func=lambda op, _: target.on(*op.qubits) if op.gate == source else op,
      deep = context.deep if context else False,
      tags_to_ignore=context.tags_to_ignore if context else ()
  )

x_y_x = [cirq.X(q), cirq.Y(q), cirq.X(q).with_tags("ignore")]
circuit = cirq.Circuit(x_y_x, cirq.CircuitOperation(cirq.FrozenCircuit(x_y_x)), x_y_x)
context = cirq.TransformerContext(deep=True, tags_to_ignore=("ignore",))
print("Original Circuit:", circuit, "\n", sep="\n")
print("Substituted Circuit:", substitute_gate_using_primitives(circuit, context=context), "\n", sep="\n")

## Analytical Gate Decompositions

Gate decomposition is the process of implementing / decomposing a given unitary `U` using only gates that belong to a specific target gateset. 

Cirq provides many analytical decomposition methods, often based on [KAK Decomposition](https://arxiv.org/abs/quant-ph/0507171), to decompose two qubit unitaries into specific target gatesets. Some notable decompositions are:

* **cirq.single_qubit_matrix_to_pauli_rotations**: Decomposes a single qubit matrix to ZPow/XPow/YPow rotations. 
* **cirq.single_qubit_matrix_to_phased_x_z**: Decomposes a single-qubit matrix to a PhasedX and Z gate.
* **cirq.two_qubit_matrix_to_sqrt_iswap_operations**: Decomposes any two-qubit unitary matrix into ZPow/XPow/YPow/sqrt-iSWAP gates.
* **cirq.two_qubit_matrix_to_cz_operations**: Decomposes any two-qubit unitary matrix into ZPow/XPow/YPow/CZ gates.
* **cirq.three_qubit_matrix_to_operations**: Decomposes any three-qubit unitary matrix into CZ/CNOT and single qubit rotations.



You can use these analytical decomposition methods to build transformers which can rewrite a given circuit using only gates from the target gateset. 

In [ ]:
@cirq.transformer
def convert_to_cz_target(
    circuit: cirq.AbstractCircuit, 
    *, 
    context: Optional[cirq.TransformerContext] = None, 
    atol: float = 1e-8,
    allow_partial_czs: bool = True
  ) -> cirq.Circuit:
  """Transformer to rewrite the given circuit using CZs + 1-qubit rotations.

  Note that the transformer decomposes only operations on <= 2-qubits and is
  presented as an illustration of using transformer primitives + analytical 
  decomposition methods. 
  """
  def map_func(op: cirq.Operation, _) -> cirq.OP_TREE:
    if not (cirq.has_unitary(op) and cirq.num_qubits(op) <= 2):
      return op
    mat = cirq.unitary(op)
    q = op.qubits
    if cirq.num_qubits(op) == 1:
      g = cirq.single_qubit_matrix_to_phxz(mat)
      return g.on(*q) if g else []
    return cirq.two_qubit_matrix_to_cz_operations(*q, mat, allow_partial_czs=allow_partial_czs, atol=atol)
    
  return cirq.map_operations_and_unroll(
      circuit, 
      map_func,
      deep=context.deep if context else False,
      tags_to_ignore=context.tags_to_ignore if context else ()
  )
circuit = cirq.testing.random_circuit(qubits=4, n_moments=8, op_density=0.8, random_state=1234)
converted_circuit = convert_to_cz_target(circuit)
cirq.testing.assert_circuits_with_terminal_measurements_are_equivalent(circuit, converted_circuit)
print(f"Circuit Depth:{len(circuit)}", circuit, sep="\n")
print(f"Circuit Depth:{len(converted_circuit)}", converted_circuit, sep="\n")

## Heuristic Gate Decompositions
Cirq also provides heuristic methods for decomposing any two qubit unitary matrix in terms of any specified two qubit target unitary + single qubit rotations. These methods are useful when accurate analytical decompositions for the target unitary are not known or when gate decomposition fidelity (i.e. accuracy of decomposition) can be traded off against decomposition depth (i.e. number of 2q gates in resulting decomposition) to achieve a higher overall gate fidelity. 


See the following resources for more details on heuristic gate decomposition:

* **`cirq.two_qubit_gate_product_tabulation`**
* **https://arxiv.org/pdf/2106.15490.pdf**

# Built-in Transformers in Cirq

## Overview
Transformers that come with cirq can be found in the `cirq.transformers` package.

A few notable examples are:
*   **cirq.align_left / cirq.align_right**: Align gates to the left/right of the circuit.
*   **cirq.defer_measurements**:  Moves all (non-terminal) measurements in a circuit to the end of circuit by implementing the deferred measurement principle.
*   **cirq.drop_empty_moments** / **cirq.drop_negligible_operations**:  Removes moments that are empty or operations that have very small effects, respectively.
*   **cirq.eject_phased_paulis**: Pushes X, Y, and PhasedX gates towards the end of the circuit, potentially absorbing Z gates and modifying gates along the way.
*   **cirq.eject_z**:  Pushes Z gates towards the end of the circuit, potentially adjusting phases of gates that they pass through.
*   **cirq.expand_composite**:  Uses `cirq.decompose` to expand composite gates.
*   **cirq.merge_k_qubit_unitaries**: Replaces connected components of unitary operations, acting on <= k qubits, with op-tree given by `rewriter(circuit_op)`.
*   **cirq.optimize_for_target_gateset**: Attempts to convert a circuit into and equivalent circuit using only gates from a given target gateset.
*   **cirq.stratified_circuit:**: Repacks the circuit to ensure that moments only contain operations from the same category.
*   **cirq.synchronize_terminal_measurements**:  Moves all terminal measurements in a circuit to the final moment, if possible.

## Compiling to NISQ targets: `cirq.CompilationTargetGateset`
Cirq's philosophy on compiling circuits for execution on a NISQ target device or simulator is that it would often require running only a handful of individual compilation passes on the input circuit, one of the other. 

**`cirq.CompilationTargetGateset`** is an abstraction in Cirq to represent such compilation targets as well as the bundles of transformer passes which should be executed to compile a circuit to this target. Concretely speaking, a compilation target gateset is:
* A subclass of `cirq.Gateset`, which can be used for description and validation of gates supported by a specific target device / simulator.
* A bundle of pre/post transformers + an analytical decomposition method that can be used to rewrite an optimized equivalent circuit using gates from this target gateset. 

Cirq has implementations for common target gatesets like `cirq.CZTargetGateset`, `cirq.SqrtIswapTargetGateset` etc.


**`cirq.optimize_for_target_gateset`** is a transformer which compiles a given circuit for a `cirq.CompilationTargetGateset` gateset via the following steps:

1. Run all `gateset.preprocess_transformers`
2. Convert operations using built-in `cirq.decompose` + `gateset.decompose_to_target_gateset`.
3. Run all `gateset.postprocess_transformers`


The preprocess transformers often includes optimizations like merging connected components of 1/2 qubit unitaries into a single unitary matrix, which can then be replaced with an efficient analytical decomposition as part of step-2. 

The post-process transformers often includes cleanups and optimizations like dropping negligible operations, converting single qubit rotations into desired form, circuit alignments etc. 

In [ ]:
context = cirq.TransformerContext(logger=cirq.TransformerLogger())
converted_circuit_optimized = cirq.optimize_for_target_gateset(circuit, gateset = cirq.CZTargetGateset(allow_partial_czs=True), context=context)
cirq.testing.assert_circuits_with_terminal_measurements_are_equivalent(circuit, converted_circuit_optimized)
print(f"Circuit Depth Original: {len(circuit)}", circuit, sep="\n")
print(f"Circuit Depth Converted Custom: {len(converted_circuit)}", converted_circuit, sep="\n")
print(f"Circuit Depth Converted Optimized: {len(converted_circuit_optimized)}", converted_circuit_optimized, sep="\n")

In [ ]:
print("Inspecting actions of individual transformers when running `cirq.optimized_for_target_gateset`\n\n")
context.logger.show()

# Summary
Cirq provides a flexible and powerful framework to
* Use built-in transformer primitives and analytical tools to create powerful custom transformers AND 
* Easily integrate custom transformers with built-in infrastructure to augment functionality like automated logging, recursive execution on sub-circuits, support for no-compile tags etc.

Cirq also provides a plethora of built-in transformers which can be composed together into a sensible abstraction, like `cirq.CompilationTargetGateset`, which in-turn can be serialized and can be used as a parameter in larger compilation pipelines and experiment workflows. 